# Extraction of Document Features

In [105]:
import ir_datasets
from tqdm import tqdm

dataset = ir_datasets.load('msmarco-document-v2')

for doc in tqdm(dataset.docs_iter()):
    print(doc)
    break

0it [00:00, ?it/s]

MsMarcoV2Document(doc_id='msmarco_doc_00_0', url='http://0-60.reviews/0-60-times/', title='0-60 Times - 0-60 | 0 to 60 Times & 1/4 Mile Times | Zero to 60 Car Reviews', headings='0-60 Times\n0-60 Times', body='0-60 Times - 0-60 | 0 to 60 Times & 1/4 Mile Times | Zero to 60 Car Reviews\n0-60 Times\nThere are many ways to measure the power a vehicle has – top speed, horsepower, foot-pounds of torque. Those are all important, but the most asked question is, “What’s the 0-60 time?” This is nothing more than a measure of how quickly a vehicle can reach the 60 mile per hour mark. It is a measure of acceleration of a vehicle. 0-60 times differ a great deal depending on the amount of power a motor puts out, of course. But anyone who spends any amount of time with car enthusiasts are sure to hear the ubiquitous term bantered around more often than most other metrics by which cars are measured in terms of power.\nThe only other measure that comes close as far as how acceleration is commonly meas

In [9]:
dataset.docs_store().get('msmarco_doc_00_0')

MsMarcoV2Document(doc_id='msmarco_doc_00_0', url='http://0-60.reviews/0-60-times/', title='0-60 Times - 0-60 | 0 to 60 Times & 1/4 Mile Times | Zero to 60 Car Reviews', headings='0-60 Times\n0-60 Times', body='0-60 Times - 0-60 | 0 to 60 Times & 1/4 Mile Times | Zero to 60 Car Reviews\n0-60 Times\nThere are many ways to measure the power a vehicle has – top speed, horsepower, foot-pounds of torque. Those are all important, but the most asked question is, “What’s the 0-60 time?” This is nothing more than a measure of how quickly a vehicle can reach the 60 mile per hour mark. It is a measure of acceleration of a vehicle. 0-60 times differ a great deal depending on the amount of power a motor puts out, of course. But anyone who spends any amount of time with car enthusiasts are sure to hear the ubiquitous term bantered around more often than most other metrics by which cars are measured in terms of power.\nThe only other measure that comes close as far as how acceleration is commonly meas

In [13]:
def length_in_tokens(string):
    return len(string.replace('\\s+', ' ').split(' '))

count_slashes_in_url('http://0-60.reviews/0-60-times/')

2

In [12]:
def count_slashes_in_url(url):
    from urllib.parse import urlparse
    return urlparse(url).path.count('/')

def extract_host_reversed(url):
    from urllib.parse import urlparse
    return '.'.join(reversed(urlparse(url).hostname.split('.')))

def dots_in_host(url):
    from urllib.parse import urlparse
    return urlparse(url).hostname.count('.')

print(count_slashes_in_url('http://fonts.googleapis.com/0-60-times/'))
(dots_in_host('http://fonts.googleapis.com/0-60-times/'))

2


2

In [31]:
from tqdm import tqdm
hosts_in_ms_marco = set()

for doc in tqdm(dataset.docs_iter()):
    hosts_in_ms_marco.add(extract_host_reversed(doc.url))

11959635it [06:21, 31358.04it/s]


In [15]:
import pandas as pd
df_alexa = pd.read_csv('/mnt/ceph/storage/data-in-progress/data-research/web-search/TREC-21/document-features/alexa-top-1m-wayback-machine-20191115000004.csv.zip', names=['rank', 'domain'])
df_alexa['dots'] = df_alexa['domain'].apply(lambda i: i.count('.'))
df_alexa

,rank,domain,dots
0,1,google.com,1
1,2,youtube.com,1
2,3,tmall.com,1
3,4,baidu.com,1
4,5,qq.com,1
...,...,...,...
899849,899850,vk7577.com,1
899850,899851,vtegroup.com,1
899851,899852,whoswho.mt,1
899852,899853,yimgun.com,1


In [16]:
df_alexa[df_alexa['dots'] > 2]

,rank,domain,dots
878,879,folha.uol.com.br,3
1041,1042,bol.uol.com.br,3
1434,1435,pagseguro.uol.com.br,3
1534,1535,noticias.uol.com.br,3
1935,1936,brasilescola.uol.com.br,3
...,...,...,...
892265,892266,washougal.k12.wa.us,3
893841,893842,familycourt.wa.gov.au,3
895443,895444,bethel.k12.or.us,3
895721,895722,ostroleka.so.gov.pl,3


In [17]:
def parse_cc_hostpath_record(line):
    line = line.split('\t')
    if len(line) != 5:
        raise ValueError('Could not handle ' + line)
    
    return {'host': line[4].strip(), 'pageRank': float(line[3].strip()), 'harmonicMean': float(line[1].strip())}


parse_cc_hostpath_record('8	2.7496426E7	10	0.001126341255286171	com.googleapis.ajax')

{'host': 'com.googleapis.ajax',
 'pageRank': 0.001126341255286171,
 'harmonicMean': 27496426.0}

In [21]:
len(hosts_in_ms_marco)

460770

In [18]:
parse_cc_hostpath_record('174489	1.7957228E7	316900	4.2665244773258744E-8	org.wikipedia.sco')['host'] in hosts_in_ms_marco

True

In [22]:
from tqdm import tqdm
import gzip
df_cc = []

with gzip.open('/mnt/ceph/storage/data-in-progress/data-research/web-search/TREC-21/document-features/cc-main-2019-aug-sep-oct-host-ranks.txt.gz','rt') as f:
    for l in tqdm(f):
        if '#harmonicc_pos	#harmonicc_val' in l:
            continue
        l = parse_cc_hostpath_record(l)
        if l['host'] in hosts_in_ms_marco:
            df_cc += [l]


820304245it [32:23, 422064.69it/s]


In [23]:
len(df_cc)

185440

In [61]:
len(df_cc)

485307

In [63]:
len(df_cc)/len(hosts_in_ms_marco)

0.4447865046407253

In [24]:
len(df_cc)/len(hosts_in_ms_marco)

0.4024567571673503

In [25]:
df_alexa_dic = {}
for _, i in df_alexa.iterrows():
    df_alexa_dic[i['domain']]= i['rank']
    
print(df_alexa_dic['google.com'])
print(df_alexa_dic['wikipedia.org'])

1
10


In [26]:
def cc_entry(doc):
    host = extract_host_reversed(doc.url)
    
    if host in df_cc_dict:
        return df_cc_dict[host]
    else:
        return None

def harmonic_mean(doc):
    ret = cc_entry(doc)
    return ret['harmonicMean'] if ret is not None else 0


def pagerank(doc):
    ret = cc_entry(doc)
    return ret['pageRank'] if ret is not None else 0

def alexa_rank(doc):
    from urllib.parse import urlparse
    host = urlparse(doc.url).hostname
    
    if host in df_alexa_dic:
        return df_alexa_dic[host]
    
    if host.count('.') >= 2:
        host = '.'.join(host.split('.')[1:])
        if host in df_alexa_dic:
            return df_alexa_dic[host]

    if host.count('.') >= 2:
        host = '.'.join(host.split('.')[1:])
        if host in df_alexa_dic:
            return df_alexa_dic[host]

    if host.count('.') >= 2:
        host = '.'.join(host.split('.')[1:])
        if host in df_alexa_dic:
            return df_alexa_dic[host]
    
    return 1000000

In [27]:
df_cc_dict = {i['host']: i for i in df_cc}

In [28]:
def calculate_document_features(doc):
    return {
        'doc_id': doc.doc_id,
        'url': doc.url,
        'feature_url_length': len(doc.url),
        'feature_slashes_in_url': count_slashes_in_url(doc.url),
        'feature_dots_in_host': dots_in_host(doc.url),
        'feature_body_length': length_in_tokens(doc.body),
        'feature_title_length': length_in_tokens(doc.title),
        'feature_pagerank': pagerank(doc),
        'feature_harmonic_mean': harmonic_mean(doc),
        'feature_alexa_rank': alexa_rank(doc)
    }

In [94]:
calculate_document_features(dataset.docs_store().get('msmarco_doc_01_169961418'))

{'doc_id': 'msmarco_doc_01_169961418',
 'url': 'http://en.wikipedia.org/WIKI/11/22/63',
 'feature_url_length': 37,
 'feature_slashes_in_url': 4,
 'feature_dots_in_host': 2,
 'feature_body_length': 5024,
 'feature_title_length': 3,
 'feature_pagerank': 0.00012952763541606714,
 'feature_harmonic_mean': 24846808.0,
 'feature_alexa_rank': 10}

In [102]:
calculate_document_features(dataset.docs_store().get('msmarco_doc_01_170251445'))

{'doc_id': 'msmarco_doc_01_170251445',
 'url': 'http://en.wikipedia.org/wiki/-eaux',
 'feature_url_length': 34,
 'feature_slashes_in_url': 2,
 'feature_dots_in_host': 2,
 'feature_body_length': 634,
 'feature_title_length': 3,
 'feature_pagerank': 0.00012952763541606714,
 'feature_harmonic_mean': 24846808.0,
 'feature_alexa_rank': 10}

In [104]:
calculate_document_features(dataset.docs_store().get('msmarco_doc_01_153290856'))

{'doc_id': 'msmarco_doc_01_153290856',
 'url': 'http://en-gb.facebook.com/help/103859036372845/',
 'feature_url_length': 47,
 'feature_slashes_in_url': 3,
 'feature_dots_in_host': 2,
 'feature_body_length': 238,
 'feature_title_length': 14,
 'feature_pagerank': 3.9621851258594565e-06,
 'feature_harmonic_mean': 20792880.0,
 'feature_alexa_rank': 7}

In [106]:
calculate_document_features(dataset.docs_store().get('msmarco_doc_00_0'))

{'doc_id': 'msmarco_doc_00_0',
 'url': 'http://0-60.reviews/0-60-times/',
 'feature_url_length': 31,
 'feature_slashes_in_url': 2,
 'feature_dots_in_host': 1,
 'feature_body_length': 816,
 'feature_title_length': 19,
 'feature_pagerank': 1.0878043580741332e-09,
 'feature_harmonic_mean': 14236359.0,
 'feature_alexa_rank': 1000000}

# Construction of the Dataset

In [107]:
import ir_datasets

dataset = ir_datasets.load('msmarco-document-v2')
df_dataset = []

for doc in tqdm(dataset.docs_iter()):
    df_dataset += [calculate_document_features(doc)]

11959635it [22:28, 8871.86it/s] 


In [108]:
df_calculated_features = pd.DataFrame(df_dataset)
df_calculated_features.to_json('/mnt/ceph/storage/data-in-progress/data-research/web-search/TREC-21/document-features/ms-marco-document-features.jsonl', orient='records', lines=True)

# MS-Marco V1 Feature Extraction

In [6]:
import ir_datasets
from tqdm import tqdm

dataset = ir_datasets.load('msmarco-document')

for doc in tqdm(dataset.docs_iter()):
    print(doc)
    break

0it [00:00, ?it/s]

MsMarcoDocument(doc_id='D1555982', url='https://answers.yahoo.com/question/index?qid=20071007114826AAwCFvR', title='The hot glowing surfaces of stars emit energy in the form of electromagnetic radiation.?', body='Science & Mathematics Physics\nThe hot glowing surfaces of stars emit energy in the form of electromagnetic radiation.?\nIt is a good approximation to assume that the emissivity e is equal to 1 for these surfaces.\nFind the radius of the star Rigel, the bright blue star in the constellation Orion that radiates energy at a rate of 2.7 x 10^32 W and has a surface temperature of 11,000 K. Assume that the star is spherical.\nUse σ =... show more\nFollow 3 answers\nAnswers\nRelevance\nRating\nNewest\nOldest\nBest Answer: Stefan-Boltzmann law states that the energy flux by radiation is proportional to the forth power of the temperature: q = ε · σ · T^4 The total energy flux at a spherical surface of Radius R is Q = q·π·R² = ε·σ·T^4·π·R² Hence the radius is R = √ ( Q / (ε·σ·T^4·π) ) 

In [10]:
dataset.docs_store().get('D1555982')

MsMarcoDocument(doc_id='D1555982', url='https://answers.yahoo.com/question/index?qid=20071007114826AAwCFvR', title='The hot glowing surfaces of stars emit energy in the form of electromagnetic radiation.?', body='Science & Mathematics Physics\nThe hot glowing surfaces of stars emit energy in the form of electromagnetic radiation.?\nIt is a good approximation to assume that the emissivity e is equal to 1 for these surfaces.\nFind the radius of the star Rigel, the bright blue star in the constellation Orion that radiates energy at a rate of 2.7 x 10^32 W and has a surface temperature of 11,000 K. Assume that the star is spherical.\nUse σ =... show more\nFollow 3 answers\nAnswers\nRelevance\nRating\nNewest\nOldest\nBest Answer: Stefan-Boltzmann law states that the energy flux by radiation is proportional to the forth power of the temperature: q = ε · σ · T^4 The total energy flux at a spherical surface of Radius R is Q = q·π·R² = ε·σ·T^4·π·R² Hence the radius is R = √ ( Q / (ε·σ·T^4·π) ) 

In [14]:
from tqdm import tqdm
hosts_in_ms_marco = set()

for doc in tqdm(dataset.docs_iter()):
    hosts_in_ms_marco.add(extract_host_reversed(doc.url))

3213835it [01:31, 35239.44it/s]


In [29]:
calculate_document_features(dataset.docs_store().get('D1555982'))

{'doc_id': 'D1555982',
 'url': 'https://answers.yahoo.com/question/index?qid=20071007114826AAwCFvR',
 'feature_url_length': 66,
 'feature_slashes_in_url': 2,
 'feature_dots_in_host': 2,
 'feature_body_length': 279,
 'feature_title_length': 14,
 'feature_pagerank': 1.1274305354321944e-06,
 'feature_harmonic_mean': 21056770.0,
 'feature_alexa_rank': 11}

In [30]:
import ir_datasets

df_dataset = []

for doc in tqdm(dataset.docs_iter()):
    df_dataset += [calculate_document_features(doc)]

3213835it [05:32, 9676.65it/s] 


In [31]:
df_calculated_features = pd.DataFrame(df_dataset)
df_calculated_features

,doc_id,url,feature_url_length,feature_slashes_in_url,feature_dots_in_host,feature_body_length,feature_title_length,feature_pagerank,feature_harmonic_mean,feature_alexa_rank
0,D1555982,https://answers.yahoo.com/question/index?qid=2...,66,2,2,279,14,1.127431e-06,21056770.0,11
1,D301595,http://childparenting.about.com/od/physicalemo...,109,4,2,889,6,1.136189e-08,17037554.0,10437
2,D1359209,http://visihow.com/Check_for_Lice_Nits,38,1,1,2573,4,2.064811e-08,18128880.0,42420
3,D2147834,http://www.nytimes.com/2010/01/05/business/glo...,62,6,2,1175,7,1.254738e-09,15542113.0,94
4,D1568809,http://www.realtor.com/realestateandhomes-sear...,69,3,2,150,8,1.032820e-09,13069215.0,522
...,...,...,...,...,...,...,...,...,...,...
3213830,D2027345,http://www.waspbarcode.com/buzz/organized-tips...,79,3,2,515,10,0.000000e+00,0.0,1000000
3213831,D3094124,http://www.simplygoodcode.com/2014/04/angularj...,95,4,2,1130,7,0.000000e+00,0.0,1000000
3213832,D1659457,http://www.ushistory.org/us/11.asp,34,2,2,821,4,0.000000e+00,0.0,8623
3213833,D1633167,http://www.symbiote.com/,24,1,2,29,5,0.000000e+00,0.0,1000000


In [32]:
df_calculated_features.to_json('/mnt/ceph/storage/data-in-progress/data-research/web-search/TREC-21/marco-v1-document-features/ms-marco-v1-document-features.jsonl', orient='records', lines=True)